# Face Recognition

**Note**: Here, we use channels-first notation of shape $(m, n_C, n_H, n_W)$.

## Packages

In [23]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Encode Face Images into a 128-dimensional Vector 

### 1.1 - Use a ConvNet  to Compute Encodings

**`FRmodel Overview`**:

`Load pre-trained weights that follow the Inception model to compute encodings.`

- **`Input`**`: face images of shape` $(m, n_C, n_H, n_W) = (m, 3, 96, 96)$
- **`Output`**`: encodings of each face image of shape` $(m, 128)$

In [24]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


### 1.2 - The Triplet Loss

Training will use triplets of images $(A, P, N)$ that are picked from the training dataset:  

- **A**nchor = picture of a person 
- **P**ositive = a picture of the same person
- **N**egative = a picture of a different person

The **triplet loss function**:
- minimizes the distance between Anchor & Positive
- maximizes the distance between Anchor & Negative 

Ensure that $A^{(i)}$ is closer to $P^{(i)}$ than to $N^{(i)}$) by at least a margin $\alpha$:

$$\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 + \alpha < \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2$$

To do this, we minimize the following **triplet cost**:

$$\mathcal{J} = \sum^{m}_{i=1} \large[ \small \underbrace{\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2}_\text{(1)} - \underbrace{\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2}_\text{(2)} + \alpha \large ] \small_+ \tag{3}$$

**Note:** Most implementations rescale encoding vectors to have L2 norm = one (i.e., $\mid \mid f(img)\mid \mid_2$=1).

In [25]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)   # compute distance between A and P
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)   # compute distance between A and N
    basic_loss = pos_dist - neg_dist + alpha                                      # subtract distances and add alpha
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0))                               # take max of basic_loss & 0, sum over examples
    
    return loss

## 2. Load Pre-Trained Model

In [26]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

## 3. Apply the Model

### 3.1 - Face Verification

Build a database containing one encoding vector for each person, generating the encoding through `img_to_encoding(image_path, model)`.

In [27]:
database = {}
database["danielle"] = img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = img_to_encoding("images/younes.jpg", FRmodel)
database["tian"] = img_to_encoding("images/tian.jpg", FRmodel)
database["andrew"] = img_to_encoding("images/andrew.jpg", FRmodel)
database["kian"] = img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = img_to_encoding("images/arnaud.jpg", FRmodel)

**`verify() Overview`**:

`verify()` checks if the front-door camera picture (`image_path`) is actually the person "identity" on the ID card.

`image_path = path to an image
identity = string, name of the person you'd like to verify the identity. Has to be an employee who works in the office.
database = python dictionary mapping names of allowed people's names (strings) to their encodings (vectors).
model = Inception model instance in Keras`

In [28]:
def verify(image_path, identity, database, model):
    
    encoding = img_to_encoding(image_path, model)                  # compute image encoding
    dist = np.linalg.norm(encoding-database[identity])                      # compute distance with identity

    if dist < 0.7:                                                 # open / close the door
        print("It's " + str(identity) + ", welcome in!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = False
        
    return dist, door_open

In [29]:
print(verify("images/camera_0.jpg", "younes", database, FRmodel))
print(verify("images/camera_2.jpg", "kian", database, FRmodel))

It's younes, welcome in!
(0.65939289, True)
It's not kian, please go away
(0.86224014, False)


### 3.2 - Face Recognition

 **`who_is_it() Overview`**:

`who_is_it() figures out if an input image is one of the authorized persons, and if so, who.`

```
Arguments:
image_path = path to image
database = dictionary of image encodings and names
model = Inception model instance in Keras

Returns:
min_dist = minimum distance between image_path encoding and database encodings
identity = name prediction for person on image_path
```

In [30]:
def who_is_it(image_path, database, model):
    
    encoding = img_to_encoding(image_path, model)         # compute image encoding
    min_dist = 100                                        # initialize min_dist to a large value
    
    for (name, db_enc) in database.items():                       # loop over database names and encodings
        dist = np.linalg.norm(encoding-db_enc)            # compute distance between encoding and current db_enc
        if dist < min_dist:                               # update min_dist and identity
            min_dist = dist
            identity = name

    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

In [31]:
print(who_is_it("images/camera_0.jpg", database, FRmodel))     # younes
print(who_is_it("images/camera_1.jpg", database, FRmodel))     # bertrand

it's younes, the distance is 0.659393
(0.65939289, 'younes')
it's bertrand, the distance is 0.467681
(0.46768054, 'bertrand')
